In [40]:
import os
import pandas as pd 
from bs4 import BeautifulSoup
from io import StringIO


In [41]:
SCORE_DIR = 'data/scores'

In [42]:
box_scores = os.listdir(SCORE_DIR)

In [43]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith('.html')]

In [44]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select('tr.over_header')]
    [s.decompose() for s in soup.select('tr.thead')]
    return soup

In [45]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs={'id' : 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols
    
    line_score = line_score[['team', 'total']]
    return line_score

In [56]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs = {'id' : f'box-{team}-game-{stat}'}, index_col= 0)[0]
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

In [72]:
box_score = box_scores[0]
soup = parse_html(box_score)
line_score = read_line_score(soup)
teams = list(line_score['team'])

summaries = []
for team in teams:
    basic = read_stats(soup, team, 'basic')
    advanced = read_stats(soup, team, 'advanced')
    
    totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
    totals.index = totals.index.str.lower()
    
    maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
    maxes.index = maxes.index.str.lower() + '_max'

In [74]:
maxes

mp_max          NaN
fg_max        8.000
fga_max      20.000
fg%_max       1.000
3p_max        2.000
3pa_max       8.000
3p%_max       0.500
ft_max        6.000
fta_max       7.000
ft%_max       1.000
orb_max       7.000
drb_max      10.000
trb_max      17.000
ast_max       6.000
stl_max       3.000
blk_max       2.000
tov_max       2.000
pf_max        4.000
pts_max      18.000
+/-_max      11.000
mp_max          NaN
ts%_max       0.862
efg%_max      1.000
3par_max      0.500
ftr_max       3.000
orb%_max     14.600
drb%_max     34.300
trb%_max     22.000
ast%_max     31.000
stl%_max      4.900
blk%_max      4.400
tov%_max     22.800
usg%_max     29.000
ortg_max    178.000
drtg_max    111.000
bpm_max       5.500
dtype: float64